**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *


[12:42:53] Initializing Normalizer
/home/hamza/mambaforge-pypy3/envs/dockm8/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/home/hamza/mambaforge-pypy3/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/hamza/mambaforge-pypy3/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated

### Data input, then create each path 
- suitable for every machine

In [2]:
protein_name = 'protein_protoss_noligand.pdb'
ligand_library = 'ecft_scores.sdf'
reference_ligand = 'ref_ligand.pdb'

### Global Path for every local machine

In [3]:
from pathlib import Path

HERE = Path(_dh[-1])
DATA = (HERE / "data")


# Move input data (protein pdb, docking library and reference ligand) to data directory
software = str(HERE / "software")
protein_file = str(DATA / protein_name)
docking_library = str(DATA / ligand_library)
ref_file = str(DATA / reference_ligand)

### DockM8 config

In [4]:
pocket = 'reference'
protonation = 'pkasolver'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', 'bestpose', 'bestpose_PLANTS', 'bestpose_GNINA', 'bestpose_SMINA']
clustering_method = 'KMedoids'
rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'RTMScore', 'vinardo', 'SCORCH']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
ncpus = 14
#Create a temporary folder for all further calculations
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
(DATA/'temp').mkdir(exist_ok=True)

The working directory has been set to: /home/hamza/Gitlab/DockM8/data


In [5]:
# if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 8)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')


[2023-Jun-10 12:43:03]: Extracting pocket from /home/hamza/Gitlab/DockM8/data/protein_protoss_noligand.pdb using /home/hamza/Gitlab/DockM8/data/ref_ligand.pdb as reference ligand

[2023-Jun-10 12:43:23]: Finished extracting pocket from /home/hamza/Gitlab/DockM8/data/protein_protoss_noligand.pdb using /home/hamza/Gitlab/DockM8/data/ref_ligand.pdb as reference ligand
{'center': [67.84, 79.11, 89.67], 'size': [16.0, 16.0, 16.0]}


In [6]:
if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)


[2023-Jun-10 12:43:23]: Standardizing docking library using ChemBL Structure Pipeline...


Standardizing molecules: 100%|██████████| 212/212 [00:00<00:00, 369.83mol/s]



[2023-Jun-10 12:43:24]: Standardization of compound library finished: Started with 212, ended with 212 : 0 compounds lost

[2023-Jun-10 12:43:24]: Calculating protonation states using pkaSolver...


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Nc1nnc(-c2cccc(OC(F)(F)F)c2)o1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Nc1nnc(-c2cccc(OC(F)(F)F)c2)o1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: Nc1nnc(-c2cccc(OC(F)(F)F)c2)o1


...

CCCCC(=O)N(Cc1ccc(-c2ccccc2-c2nnn[nH]2)cc1)[C@H](C(=O)O)C(C)C

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

CCCCC(=O)N(Cc1ccc(-c2ccccc2-c2nnn[nH]2)cc1)[C@H](C(=O)O)C(C)C

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: CCCCC(=O)[NH+](Cc1ccc(-c2ccccc2-c2nnn[nH]2)cc1)[C@H](C(=O)[O-])C(C)C


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Cc1ccc(-c2noc(CN3CC=C(c4c[nH]c5ccccc45)CC3)n2)cc1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Cc1ccc(-c2noc(CN3CC=C(c4c[nH]c5ccccc45)CC3)n2)cc1

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: Cc1ccc(-c2noc(C[NH+]3CC=C(c4c[nH]c5ccccc45)CC3)n2)cc1


[query.py:297 - calculate_microstate_pka_values()] Using dimorphite-dl to identify protonation sites.


Cn1ncc2c(=O)[nH]c(Nc3cccc(Oc4ccccc4)c3)nc21

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Cn1ncc2c(=O)[nH]c(Nc3cccc(Oc4ccccc4)c3)nc21

For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.

Proposed mol at pH 7.4: Cn1ncc2c(=O)[nH]c(Nc3cccc(Oc4ccccc4)c3)nc21

[2023-Jun-10 12:51:13]: Protonation of compound library finished: Started with 212, ended with 212 : 0 compounds lost

[2023-Jun-10 12:51:13]: Generating 3D conformers using GypsumDL...

[2023-Jun-10 12:52:04]: Cl

In [7]:
docking(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus, pocket_definition)


Splitting SDF file final_library.sdf ...


Splitting files: 100%|██████████| 27/27 [00:00<00:00, 267.44it/s]


Split docking library into 27 files each containing 8 compounds

[2023-Jun-10 12:52:04]: Combining all poses...

[2023-Jun-10 12:52:06]: All poses succesfully combined!


In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 20)


In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

In [10]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'RTMScore'])

The folder: /home/hamza/Gitlab/DockM8/data/temp/ranking was created


FileNotFoundError: [Errno 2] No such file or directory: '/home/hamza/Gitlab/DockM8/data/temp/rescoring_bestpose_clustered/allposes_rescored.csv'